In [1]:
import cv2
import numpy as np
import tensorflow as tf
import dlib
from pygame import mixer
import time

pygame 2.0.1 (SDL 2.0.14, Python 3.8.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
eye_model = tf.keras.models.load_model("Eye_tracking_CNN.model")
yawn_model = tf.keras.models.load_model("Yawn_Tracking_CNN.model")

In [3]:
def prepare(img):
    image_size = 100
    new_array = cv2.resize(img, (image_size, image_size))
    return new_array.reshape(-1, image_size, image_size, 1)

In [4]:
def pause(time_gap):
    clock = time.time()
    while True:
        if time.time() - clock >= time_gap:
            break

In [5]:
def play_alarm():
    mixer.init()
    mixer.music.load(r"C:\Users\Deboparna Banerjee\Music\AGUST D.mp3")
    mixer.music.set_volume(0.1)
    mixer.music.play()
    
    ## pause for 5s
    pause(5)
            
    ## stop alarm
    mixer.music.stop()

In [6]:
display_eye = {
    0 : 'Closed',
    1 : 'Open'
}

display_yawn = {
    0 : 'Yawn',
    1 : 'No Yawn'
}

In [7]:
def classifier(img, flags):
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor('shape_68.dat')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    ## feed to yawn model
    yawn_prediction = yawn_model.predict([prepare(gray)])

    faces = detector(gray)
    for face in faces:
        landmarks = predictor(gray, face)

        ## left eye mark
        x1_l = landmarks.part(36).x - 10
        y1_l = landmarks.part(38).y - 10
        x2_l = landmarks.part(39).x + 10
        y2_l = landmarks.part(40).y + 10
        ## drawing rectangle
        cv2.rectangle(img,(x1_l,y1_l),(x2_l,y2_l),(255,0,0),2)
        left_eye = gray[y1_l:y2_l, x1_l:x2_l]                          ## crop to eye
        prediction_left = eye_model.predict([prepare(left_eye)])       ## feed to model
        ## write predicition on image
        img=cv2.putText(img, display_eye[int(prediction_left[0][0])], (x1_l, y1_l-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)

        ## right eye mark
        x1_r = landmarks.part(42).x - 10
        y1_r = landmarks.part(44).y - 10
        x2_r = landmarks.part(45).x + 10
        y2_r = landmarks.part(46).y + 10
        cv2.rectangle(img,(x1_r,y1_r),(x2_r,y2_r),(255,0,0),2)
        right_eye = gray[y1_r:y2_r, x1_r:x2_r]
        prediction_right = eye_model.predict([prepare(right_eye)])
        img=cv2.putText(img, display_eye[int(prediction_right[0][0])], (x1_r, y1_r-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1, cv2.LINE_AA)
        
         
        ## Eye thresholding
        if int(prediction_right[0][0]) == int(prediction_left[0][0]) == 0:
            flags['close_frame_count'] += 1
        else:
            flags['close_frame_count'] = 0
        
        if flags['close_frame_count'] >= 6:
            img=cv2.putText(img, "ALERT!! Keep your eyes open!", (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1, cv2.LINE_AA)
            w  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))   # float `width`
            h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))  # float `height`
            img = cv2.rectangle(img, (0,0), (w,h), (0,0,255), 5)
            
        ## Yawn thresholding
        if int(yawn_prediction[0][0]) == 0:
            img=cv2.putText(img, "Yawn Detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 1, cv2.LINE_AA)
     
    return img, flags

In [11]:
cap = cv2.VideoCapture(0)
flags = {
    ## for eyes
    'close_frame_count' : 0,
    ## for yawn
    'yawn_counter' : 0,
    'yawn_frame_count' : 0,
    'time_frame_count' : 0
}

while True:
    ret, frame = cap.read()
    result_img, flag_eye = classifier(frame, flags)
    cv2.imshow('Are you drowsy?', result_img)
    if cv2.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv2.destroyAllWindows()